In [38]:
import pandas as pd
import numpy as np
from scipy.special import softmax,  expit
from scipy.stats import zscore
import ast
import os
import os.path as osp

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from matplotlib import pyplot as plt
from miga.const import ID2LABELS_SMG_SHORT as ID2LABELS
LABELS2ID = {v:k for k,v in ID2LABELS.items()}

In [39]:
path_to_folder = r'D:\Project-mpg microgesture\human_micro_gesture_classifier\prediction_over_videos'
# file_name = 'prediction_over_videos_Sample0031_color_935_2000.csv'
# file_name = 'prediction_over_videos_Sample0031_color_12400_13700.csv'
# file_name = 'prediction_over_videos_Sample0031_color_21800_22200.csv'
# file_name ='prediction_over_videos_Sample0031_color_21800_22200_new.csv'
# file_name = 'prediction_over_videos_Sample0031_color_5700_6200_new_resampled.csv'
file_name = 'prediction_over_videos_Sample0031_color_12000_14000_new_resampled.csv'
path_to_df = osp.join(path_to_folder, file_name)


# start_frame = 12400
# sample_ratio = 2
start_frame = 12000
sample_ratio = 4




df = pd.read_csv(path_to_df, header=None, index_col=None)
df.columns  = ['frame','resolution','logits','smoothed_logits','predictions','labels']

df['logits_np'] = df['logits'].apply(lambda x: np.array(ast.literal_eval(x.replace('\n','').replace(' ',',').replace(',,',',').replace(',,',',').replace(',,',','))))
# remove moving legs
# df['logits_np'] = df['logits_np'].apply(lambda x: x[[0,1,2,3,5,6,7,8,9,10,11,12,13,14,15,16]])

# df['sigmoid'] = df['logits_np'].apply(lambda x: expit(x))
df['sigmoid'] = df['logits_np'].apply(lambda x: softmax(x))

# df['sigmoid'] = df['logits_np']
# df['sigmoid'] = df['sigmoid'].apply(lambda x: zscore(x))

# df['sigmoid'] = df['sigmoid'].apply(lambda x: (x-min(x))/max(x)-min(x))
# df['sigmoid'] = (np.stack(df['sigmoid'].values) -np.stack(df['sigmoid'].values)[:,:1] -10).tolist()





df['true_frame'] = df['frame'].apply(lambda x: x * sample_ratio + start_frame)

# labels_columns = [a for a in ID2LABELS.values() if a not in ['Moving legs']]
labels_columns = ID2LABELS.values()
df_split = pd.DataFrame(df['sigmoid'].tolist(), columns=labels_columns)
# df_split = pd.DataFrame(df['sigmoid'], columns=labels_columns)
df_split['frame'] = df['true_frame']
df_split['resolution'] = df['resolution']
df_split.columns

Index(['Turtling neck', 'Rubbing face', 'Folding arms', 'Touching chest',
       'Moving legs', 'Scratching neck', 'Arms behind', 'Rubbing hands',
       'Arms akimbo', 'Crossing legs', 'Scratching body', 'Touching face',
       'Adjusting hair', 'Holding arms', 'Pulling collar', 'Playing wobjects',
       'Hand gestures', 'frame', 'resolution'],
      dtype='object')

In [40]:
x = df['logits_np'].values
a = x[0]
softmax(a)

array([0.05789837, 0.05885629, 0.05995973, 0.05634844, 0.04815831,
       0.05813514, 0.06099658, 0.06035626, 0.06741868, 0.05834965,
       0.05815675, 0.06268258, 0.0591691 , 0.06011183, 0.05716916,
       0.05810969, 0.05812343])

In [41]:
df = pd.read_csv(path_to_df, header=None, index_col=None)
df.columns  = ['frame','resolution','logits','smoothed_logits','predictions','labels']

df['logits_np'] = df['logits'].apply(lambda x: np.array(ast.literal_eval(x.replace('\n','').replace(' ',',').replace(',,',',').replace(',,',',').replace(',,',','))))
df_g = list(df[['resolution','frame','logits_np']].groupby('resolution'))
res, df_e = df_g[1]
df_temp = df_e.copy()
df_temp['sigmoid'] = df_temp['logits_np'].apply(lambda x: softmax(x))
df_temp.columns

Index(['resolution', 'frame', 'logits_np', 'sigmoid'], dtype='object')

In [42]:
LABELS2ID.values()

dict_values([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17])

In [43]:
fig = go.Figure()
labels_columns = list(LABELS2ID.keys())
frames = df_temp['frame']
y = np.stack(df_temp['sigmoid'].values)
for i in range(0,len(labels_columns)):
    fig.add_trace(go.Scatter(x=frames, y=y[:,i], mode='lines', name=labels_columns[i]))
fig.show()

In [44]:
df['sigmoid']

KeyError: 'sigmoid'

In [ ]:
x = df.iloc[0]['logits_np']
sum(softmax(x))

In [ ]:
x = df.iloc[0]['logits_np']
print(softmax(x.reshape(-1,1),axis=0).reshape(1, -1))
print(expit(x))

In [45]:
# Assuming df is your DataFrame

# Step 1: Group by "frame"
grouped = df_split.copy().groupby('frame')
ret = []
# Step 2 and 3: Calculate mean and update rows
for frame, group in grouped:
    # Calculate mean excluding rows with resolution as "aggregated"
    mean_values = group.loc[group['resolution'] != 'aggregated'][labels_columns].mean().values.reshape(1,-1)
    
    # Update rows where resolution is "aggregated" with the mean values
    aggregated_mask = (group['resolution'] == 'aggregated')
    group.loc[aggregated_mask,labels_columns] = mean_values  # Replace 'value_column' with the actual column name you want to update
    ret.append(group)
df_split_mean = pd.concat(ret)
df_split_mean.resolution.unique()

array(['aggregated', '32'], dtype=object)

In [46]:
df_split

,Turtling neck,Rubbing face,Folding arms,Touching chest,Moving legs,Scratching neck,Arms behind,Rubbing hands,Arms akimbo,Crossing legs,Scratching body,Touching face,Adjusting hair,Holding arms,Pulling collar,Playing wobjects,Hand gestures,frame,resolution
0,0.057898,0.058856,0.059960,0.056348,0.048158,0.058135,0.060997,0.060356,0.067419,0.058350,0.058157,0.062683,0.059169,0.060112,0.057169,0.058110,0.058123,12000,aggregated
1,0.057898,0.058856,0.059960,0.056348,0.048158,0.058135,0.060997,0.060356,0.067419,0.058350,0.058157,0.062683,0.059169,0.060112,0.057169,0.058110,0.058123,12000,32
2,0.057965,0.059134,0.059889,0.056243,0.047068,0.058134,0.061285,0.060172,0.067479,0.058212,0.058198,0.062809,0.059479,0.060218,0.057330,0.058212,0.058173,12004,aggregated
3,0.057965,0.059134,0.059889,0.056243,0.047068,0.058134,0.061285,0.060172,0.067479,0.058212,0.058198,0.062809,0.059479,0.060218,0.057330,0.058212,0.058173,12004,32
4,0.057781,0.060264,0.058517,0.056009,0.042817,0.058124,0.062569,0.058346,0.067731,0.057593,0.058982,0.063072,0.061646,0.060487,0.058279,0.058923,0.058860,12008,aggregated
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997,0.057447,0.049384,0.068477,0.056970,0.095538,0.057115,0.051671,0.073027,0.063027,0.062309,0.051962,0.058365,0.044510,0.056447,0.049712,0.051871,0.052167,13992,32
998,0.057426,0.049508,0.068296,0.057017,0.094857,0.057147,0.051797,0.072759,0.063113,0.062222,0.052112,0.058400,0.044745,0.056485,0.049823,0.051991,0.052302,13996,aggregated
999,0.057426,0.049508,0.068296,0.057017,0.094857,0.057147,0.051797,0.072759,0.063113,0.062222,0.052112,0.058400,0.044745,0.056485,0.049823,0.051991,0.052302,13996,32
1000,0.057415,0.049392,0.068442,0.057006,0.095450,0.057142,0.051668,0.072974,0.063050,0.062320,0.051971,0.058385,0.044535,0.056449,0.049772,0.051888,0.052143,14000,aggregated


In [47]:
def plot_traces(df_split, resolution='64'):
# resolution='64'
        # Filter the relevant row corresponding to the input "resolution"
    df_filtered = df_split[df_split['resolution'] == resolution].drop(columns=['resolution'])
    
    # Extract the values from the DataFrame
    # Extract the values from the DataFrame
    frames = df_filtered['frame']
    labels = df_filtered.loc[:, labels_columns]  # Select columns from 0 to 16
    
    
    # Create the Plotly figure
    fig = go.Figure()
    
    # Add each trace to the plot
    for i, label in enumerate(labels_columns):
        fig.add_trace(go.Scatter(x=frames, y=labels[label], mode='lines', name=f'{label}:{i+1}',
                      hovertemplate=[f'Frame: {frames.values[k]}<br>Pred:{labels[label].values[k]:.3f}' for k in range(len(frames))]))
    fig.update_traces(mode="lines+text")
    # Update layout
    fig.update_layout(title=f'Predictions Over Time:{resolution}',
                      xaxis_title='Frame Number',
                      yaxis_title='Value',
                       autosize=False,
                        width=1200,
                        height=600)
    
    #  # Show the plot
    fig.show()
plot_traces(df_split, resolution='aggregated')
# plot_traces(df_split, resolution='16')
plot_traces(df_split, resolution='32')
plot_traces(df_split, resolution='64')
# plot_traces(df_split, resolution='128')